# Packages

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
%matplotlib inline

# Dataset

In [2]:
#prescription from primary care data
scripts = pd.read_csv('../primary_care/gp_scripts.txt', sep = '\t', encoding='ISO-8859-1')

In [3]:
# previous dataset",
records = pd.read_pickle('../primary_care/full_records.pkl')

In [4]:
#bayer prescriptions
prescriptions = pd.read_excel('../primary_care/prescriptions.xlsx')

In [5]:
#drug encodings
drug_lkps = pd.read_excel('../primary_care/all_lkps_maps.xlsx', sheet_name='read_v2_drugs_lkp')

In [108]:
records

,eid,coded,data_provider,event_dt,read_2,read_3,value1,value2,value3,condition,...,first_record_d,first_record_diag,first_stroke_after_d,first_stroke_type,pri,sec,discrepancy,noRecords,noRecords2,event_int
0,1000255,"[[02/09/2015, cad, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2015-09-02,cad,NaT,NaN,[],"[[2015-09-02 00:00:00, cad, sec]]",False,1,1,NaN
1,1000421,"[[04/08/2010, cad, sec], [27/05/2010, stroke, ...","[3, 3]","[27/05/2010, 04/10/2010]","[nan, nan]","[XE0VK, XE2uV]","[nan, nan]","[nan, nan]","[nan, nan]","[TIA, CAD]",...,2010-05-27,tia,2010-05-27,stroke,"[[2010-05-27 00:00:00, tia, pri], [2010-10-04 ...","[[2010-05-27 00:00:00, stroke, sec], [2010-08-...",False,4,4,0.000000
2,1000466,"[[11/06/2002, cad, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2002-06-11,cad,NaT,NaN,[],"[[2002-06-11 00:00:00, cad, sec]]",False,1,1,NaN
3,1000584,"[[16/06/1997, cad, sec], [19/10/2004, afib, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1997-06-16,cad,NaT,NaN,[],"[[1997-06-16 00:00:00, cad, sec], [2004-10-19 ...",False,2,2,NaN
4,1000674,"[[31/05/2006, cad, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2006-05-31,cad,NaT,NaN,[],"[[2006-05-31 00:00:00, cad, sec]]",False,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53318,6020725,[],"[3, 3]","[01/01/1993, 18/09/2003]","[nan, nan]","[G66.., XE0VK]","[nan, nan]","[nan, nan]","[nan, nan]","[Stroke, TIA]",...,1993-01-01,stroke,2003-09-18,tia,"[[1993-01-01 00:00:00, stroke, pri], [2003-09-...",[],False,2,2,128.528307
53319,6021009,[],[3],[09/06/2016],[nan],[G5730],[nan],[nan],[nan],[Afib],...,2016-06-09,afib,NaT,NaN,"[[2016-06-09 00:00:00, afib, pri]]",[],False,1,1,NaN
53320,6021143,[],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[20/01/2005, 11/02/2005, 07/04/2005, 21/11/200...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[X00DW, X00DW, X00DW, X00DW, X00DW, X00DW, X00...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[TIA, TIA, TIA, TIA, TIA, TIA, TIA, TIA, TIA, ...",...,2005-01-20,tia,2005-02-11,tia,"[[2005-01-20 00:00:00, tia, pri], [2005-02-11 ...",[],False,19,19,0.722807
53321,6022795,[],[2],[19/01/2015],[G5730],[nan],[nan],[nan],[nan],[Afib],...,2015-01-19,afib,NaT,NaN,"[[2015-01-19 00:00:00, afib, pri]]",[],False,1,1,NaN


# Prescriptions

In [ ]:
prescriptions.columns = ['Antidiabetes', 'Antihyperlipidemic', 'Antihypertensive']

In [ ]:
prescription_name = {
    'antidiabetes' : {
        'names': list(prescriptions['Antidiabetes'][:15])
    },
    'antihyperlipidemic' : {
        'names': list(prescriptions['Antihyperlipidemic'][:6])
    },
    'antihypertensive' : {
        'names': list(prescriptions['Antihypertensive'].values)
    }, 
    'all' : {
        'names': list(prescriptions['Antidiabetes'][:15])+ list(prescriptions['Antihyperlipidemic'][:6]) + list(prescriptions['Antihypertensive'].values)
    }
}

# patients

In [6]:
patients = list(records['eid'])

In [7]:
len(patients)

53323

In [8]:
relevant_scripts = scripts[scripts['eid'].isin(patients)].reset_index()

In [9]:
relevant_scripts.shape

(17036267, 9)

In [10]:
relevant_scripts.columns

Index(['index', 'eid', 'data_provider', 'issue_date', 'read_2', 'bnf_code',
       'dmd_code', 'drug_name', 'quantity'],
      dtype='object')

In [11]:
concise_scripts = relevant_scripts[['eid', 'read_2', 'bnf_code', 'dmd_code', 'drug_name', 'quantity']]

In [12]:
concise_scripts['drug_name'] = concise_scripts['drug_name'].str.lower()

/Users/michelleng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
prescriptions = list(relevant_scripts.drug_name.unique())

In [14]:

prescriptions_clean = [x.replace("*", "") for x in prescriptions if type(x) == str]

In [ ]:
ointments = []
oral = []
intravenous = []
optha = []
ENT = []
equipment = []
alternative = []
supplements = []
unknown = []
accessories = ['bag', 'stocking', 'catheter', 'stockinette', 
               'dressing', 'suture', 'test', 'tape', 'bandage',
              'swab', 'syringe', 'needle', 'ostomy']
transdermal = ['injection', 'vaccine', 'hypodermic', 'inj']
nasal = ['inhaler', 'nasal spray', 'ear', 'inhalation', 'inh']
dermal = ['oint', 'ointment', 'cream', 'lotion', 'crm', 'dermal', 
          'shampoo', 'wash', 'spray', 'patches', 'gel', 
          'emollient', 'derm']
supplement = ['shake', 'supplement', 'supplemental', 'vitamin']
ingest = ['tabs', 'tablets', 'tab', 'cap','caps', 'capsule', 'oral']
suppository = ['suppository', 'pessary', 'rectal']
for x in prescriptions_clean: 
    if type(x) == float: 
        continue
    elif any(i in x for i in ingest):
        oral.append(x)
    elif any(n in x for n in nasal):
        ENT.append(x)
    elif any(d in x for d in dermal):
        ointments.append(x)
    elif any(t in x for t in transdermal):
        intravenous.append(x)
    elif 'eye' in x:
        optha.append(x)
    elif any(a in x for a in accessories):
        equipment.append(x)
    elif any(su in x for su in supplement):
        supplements.append(x)
    elif any(s in x for s in suppository):
        alternative.append(x)
    else:
        unknown.append(x)

In [ ]:
dfnames= ['ENT','ointments', 'intravenous', 'optha', 'equipment', 'oral', 'alternative','supplements', 'unknown']

In [ ]:
dfs = [ENT, ointments, intravenous, optha, equipment, oral, alternative, supplements, unknown]

In [ ]:
combined = pd.DataFrame(columns = ['prescription', 'proposedcategory'])
def dfmaker(dflist, dfnamelist, resultdf):
    for i in range(len(dflist)):
        temp = pd.DataFrame(dflist[i], columns = ['prescription'])
        temp['proposedcategory'] = dfnamelist[i]
        resultdf = resultdf.append(temp)
    return resultdf

        

In [ ]:
combined = dfmaker(dfs, dfnames, combined)


In [ ]:
a = list(combined.prescription.unique())

In [ ]:
[elem for elem in prescriptions_clean if elem not in a ]

In [ ]:
combined.shape

In [ ]:
combined['group'] = [x.split(" ")[0] if type(x.split(" ")[0]) == str else np.nan for x in combined['prescription']]

In [ ]:
len(combined.group.unique())

In [ ]:
combined.groupby('group').agg(list)

In [ ]:
combined_arranged = combined[['group', 'prescription', 'proposedcategory']]

In [ ]:
combined_arranged.to_csv('../primary_care/unique_medications.csv')

In [ ]:
oralmed = combined[combined['proposedcategory'] == 'oral']

In [ ]:
oralmed.groupby('group').agg(list)

In [268]:
list(combined.group.unique())

['flixonase',
 'betnesol-n',
 'fluticasone',
 'gentisone',
 'sterimar',
 'salbutamol',
 'becotide',
 'beclometasone',
 'heliclear',
 'qvar',
 'salamol',
 'olive',
 'cerumol',
 'sofradex',
 'beconase',
 'sodium',
 'mometasone',
 'ventolin',
 'beclazone',
 'betamethasone',
 'clenil',
 'flutiform',
 'flumetasone',
 'otomize',
 'locorten',
 'ciprofloxacin',
 'liquifilm',
 'gentamicin',
 'triamcinolone',
 'ipratropium',
 'budesonide',
 'xylometazoline',
 'rinatec',
 'pelican',
 'betnesol',
 'nasonex',
 'terbutaline',
 'serevent',
 'salmeterol',
 'avamys',
 'otosporin',
 'atrovent',
 'viscotears',
 'hylo-tear',
 'nicotine',
 'docusate',
 'urea',
 'nicorette',
 'earcalm',
 'geltears',
 'acetic',
 'tears',
 'symbicort',
 'fostair',
 'carmellose',
 'sumatriptan',
 'niquitin',
 'azelastine',
 'becloforte',
 'chloramphenicol',
 'dexamethasone',
 'clear',
 'rhinolast',
 'libra',
 'jade',
 'gb',
 'locorten-vioform',
 'sterile',
 'gb3',
 'audicort',
 'beclomethasone',
 'minims',
 'neomycin',
 'combi

In [15]:
relevant_scripts

,index,eid,data_provider,issue_date,read_2,bnf_code,dmd_code,drug_name,quantity
0,2618,1000421,3,08/12/2006,NaN,04.07.01.02.00,NaN,Co-codamol 30mg/500mg tablets,100 tab
1,2619,1000421,3,11/01/2007,NaN,04.07.01.02.00,NaN,Co-codamol 30mg/500mg tablets,100 tab
2,2620,1000421,3,02/02/2007,NaN,04.07.02.00.00,NaN,Tramadol 50mg capsules,100 cap
3,2621,1000421,3,02/02/2007,NaN,10.01.01.00.00,NaN,Diclofenac sodium 50mg gastro-resistant tablets,84 tab
4,2622,1000421,3,04/05/2007,NaN,04.07.02.00.00,NaN,Tramadol 50mg capsules,100 cap
...,...,...,...,...,...,...,...,...,...
17036262,57706536,6025040,1,05/06/2017,bxi3.00,NaN,320031002.0,Atorvastatin 40mg tablets,56.000
17036263,57706537,6025040,1,05/06/2017,d711.00,NaN,321745007.0,Amitriptyline 10mg tablets,56.000
17036264,57706538,6025040,1,05/06/2017,d711.00,NaN,321745007.0,Amitriptyline 10mg tablets,56.000
17036265,57706539,6025040,1,05/06/2017,in45.00,NaN,329266003.0,Thiamine 100mg tablets,56.000


In [16]:
relevant_scripts[relevant_scripts['drug_name'].str.contains('aspirin') == True]

,index,eid,data_provider,issue_date,read_2,bnf_code,dmd_code,drug_name,quantity
345374,1151113,1100490,1,16/10/2009,bu2B.00,NaN,3.197810e+08,aspirin enteric coated tablets 75mg,100.000
345379,1151118,1100490,1,05/11/2009,bu23.00,NaN,3.197730e+08,aspirin dispersible tablet 75mg,100.000
345382,1151121,1100490,1,04/12/2009,bu2B.00,NaN,3.197810e+08,aspirin enteric coated tablets 75mg,56.000
345393,1151132,1100490,1,01/07/2010,bu2B.00,NaN,3.197810e+08,aspirin enteric coated tablets 75mg,56.000
345398,1151137,1100490,1,31/08/2010,bu2B.00,NaN,3.197810e+08,aspirin enteric coated tablets 75mg,56.000
...,...,...,...,...,...,...,...,...,...
15986426,54216881,5726070,1,01/08/2007,bu23.00,NaN,3.197730e+08,aspirin dispersible tablet 75mg,28.000
16150908,54794247,5778574,1,02/11/1999,di1c.00,NaN,2.211500e+16,Nu-seals aspirin ec 300mg Gastro-resistant tab...,100.000
16150915,54794254,5778574,1,10/01/2000,di1c.00,NaN,2.211500e+16,Nu-seals aspirin ec 300mg Gastro-resistant tab...,112.000
16633077,56347804,5912906,1,19/01/2000,bu2A.00,NaN,2.213500e+16,Nu-seals aspirin ec 75mg Gastro-resistant tabl...,100.000


In [17]:
relevant_scripts.sort_values('bnf_code')

,index,eid,data_provider,issue_date,read_2,bnf_code,dmd_code,drug_name,quantity
6868583,23478594,3063971,3,28/10/1992,NaN,01.01.01.01.00,NaN,Maalox TC tablets (Aventis Pharma),100
13696685,46555439,5066755,3,28/01/1997,NaN,01.01.01.01.00,NaN,Magnesium trisilicate compound tablets,100 tabs
12416763,42167298,4685852,3,13/08/2010,NaN,01.01.01.01.00,NaN,Magnesium trisilicate oral suspension,1 pack of 200 mls
12416762,42167297,4685852,3,13/08/2010,NaN,01.01.01.01.00,NaN,Magnesium trisilicate oral suspension,1 pack of 200 mls
478709,1590505,1140525,3,02/06/1992,NaN,01.01.01.01.00,NaN,Magnesium trisilicate oral suspension,500 ml
...,...,...,...,...,...,...,...,...,...
17036262,57706536,6025040,1,05/06/2017,bxi3.00,NaN,320031002.0,Atorvastatin 40mg tablets,56.000
17036263,57706537,6025040,1,05/06/2017,d711.00,NaN,321745007.0,Amitriptyline 10mg tablets,56.000
17036264,57706538,6025040,1,05/06/2017,d711.00,NaN,321745007.0,Amitriptyline 10mg tablets,56.000
17036265,57706539,6025040,1,05/06/2017,in45.00,NaN,329266003.0,Thiamine 100mg tablets,56.000


In [18]:
relevant_scripts.sort_values('dmd_code')

,index,eid,data_provider,issue_date,read_2,bnf_code,dmd_code,drug_name,quantity
13930373,47230525,5124209,1,24/09/1999,NaN,NaN,0.0,HEPATYRIX VAC,1.000
5225499,17913599,2577573,1,01/03/1988,NaN,NaN,0.0,POTASSIUM PERMANGANA SOL,500.000
6007442,20577649,2811204,1,24/04/1990,NaN,NaN,0.0,LOCORTEN-VIOFORM EAR SOL,1.000
15843853,53719707,5683940,1,14/03/1990,NaN,NaN,0.0,TUBIGRIP SIZE E (8.7,2.000
5367616,18349418,2613127,1,31/01/1989,NaN,NaN,0.0,GAUZE DRESSING- BACT,2.000
...,...,...,...,...,...,...,...,...,...
17035366,57705640,6025023,4,25/08/2017,bl8i.,NaN,NaN,NaN,NaN
17035367,57705641,6025023,4,15/09/2017,ei15.,NaN,NaN,NaN,NaN
17035368,57705642,6025023,4,15/09/2017,ei15.,NaN,NaN,NaN,NaN
17035369,57705643,6025023,4,15/09/2017,ka83.,NaN,NaN,NaN,NaN


In [19]:
relevant_scripts.sort_values('bnf_code').tail(100)

,index,eid,data_provider,issue_date,read_2,bnf_code,dmd_code,drug_name,quantity
17036167,57706441,6025040,1,10/08/2015,in45.00,NaN,329266003.0,Thiamine 100mg tablets,56.000
17036168,57706442,6025040,1,10/08/2015,in72.00,NaN,329298009.0,Vitamin B compound strong tablets,112.000
17036169,57706443,6025040,1,17/08/2015,a6b5.00,NaN,317295004.0,Omeprazole 40mg gastro-resistant capsules,56.000
17036170,57706444,6025040,1,17/08/2015,bd11.00,NaN,318352004.0,Propranolol 10mg tablets,112.000
17036171,57706445,6025040,1,17/08/2015,bxi3.00,NaN,320031002.0,Atorvastatin 40mg tablets,56.000
...,...,...,...,...,...,...,...,...,...
17036262,57706536,6025040,1,05/06/2017,bxi3.00,NaN,320031002.0,Atorvastatin 40mg tablets,56.000
17036263,57706537,6025040,1,05/06/2017,d711.00,NaN,321745007.0,Amitriptyline 10mg tablets,56.000
17036264,57706538,6025040,1,05/06/2017,d711.00,NaN,321745007.0,Amitriptyline 10mg tablets,56.000
17036265,57706539,6025040,1,05/06/2017,in45.00,NaN,329266003.0,Thiamine 100mg tablets,56.000


In [20]:
concise_scripts

,eid,read_2,bnf_code,dmd_code,drug_name,quantity
0,1000421,NaN,04.07.01.02.00,NaN,co-codamol 30mg/500mg tablets,100 tab
1,1000421,NaN,04.07.01.02.00,NaN,co-codamol 30mg/500mg tablets,100 tab
2,1000421,NaN,04.07.02.00.00,NaN,tramadol 50mg capsules,100 cap
3,1000421,NaN,10.01.01.00.00,NaN,diclofenac sodium 50mg gastro-resistant tablets,84 tab
4,1000421,NaN,04.07.02.00.00,NaN,tramadol 50mg capsules,100 cap
...,...,...,...,...,...,...
17036262,6025040,bxi3.00,NaN,320031002.0,atorvastatin 40mg tablets,56.000
17036263,6025040,d711.00,NaN,321745007.0,amitriptyline 10mg tablets,56.000
17036264,6025040,d711.00,NaN,321745007.0,amitriptyline 10mg tablets,56.000
17036265,6025040,in45.00,NaN,329266003.0,thiamine 100mg tablets,56.000


In [21]:
drug_name_counts = concise_scripts.groupby('drug_name').count()['eid'].reset_index()

In [22]:
concise_scripts = concise_scripts.drop_duplicates('drug_name')

In [23]:
drug_name_counts.columns = ['drug_name', 'counts']

In [24]:
drug_name_counts.counts

0          1
1          3
2          1
3          2
4          1
        ... 
37679      9
37680     32
37681    137
37682      8
37683      2
Name: counts, Length: 37684, dtype: int64

In [25]:
unknown_meds = relevant_scripts[relevant_scripts['drug_name'].isnull() == True]

In [26]:
unknown_medsdf = unknown_meds.groupby('read_2').count().drop('index', axis = 1).reset_index()

In [27]:
unknown_medications = unknown_medsdf[['read_2', 'eid']]

In [28]:
unknown_medications.columns = ['read_2', 'count']

In [29]:
unknown_medications

,read_2,count
0,a116.,35
1,a123.,250
2,a124.,6
3,a125.,75
4,a134.,110
...,...,...
6483,sn2q.,11
6484,sn2t.,13
6485,so24.,105
6486,so26.,2


In [30]:
concise_scripts['name'] = [x.split(' ')[0] if type(x) == str else np.nan for x in concise_scripts['drug_name']]

/Users/michelleng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
bnf_scripts = concise_scripts[concise_scripts['bnf_code'].isnull() == False].sort_values('bnf_code').reset_index()

In [32]:
drug_name_counts.describe()

,counts
count,37684.000000
mean,393.702632
std,5902.766805
min,1.000000
25%,2.000000
50%,8.000000
75%,45.000000
max,593035.000000


In [33]:
bnf = pd.merge(bnf_scripts[['bnf_code', 'drug_name', 'name']], drug_name_counts, on='drug_name', how = "left")

In [34]:
bnf

,bnf_code,drug_name,name,counts
0,01.01.01.01.00,aluminium hydroxide mixture sugar-free,aluminium,15
1,01.01.01.01.00,aluminium hydroxide 475mg capsules,aluminium,30
2,01.01.01.01.00,co-magaldrox 300mg/600mg tablets,co-magaldrox,1
3,01.01.01.01.00,maalox tc tablets (aventis pharma),maalox,8
4,01.01.01.01.00,co-magaldrox 300mg/600mg/5ml oral suspension s...,co-magaldrox,16
...,...,...,...,...
22968,A70601,glutafin select gluten-free part-baked white ...,glutafin,23
22969,A7060101,glutafin gluten-free wheat free multipurpose ...,glutafin,5
22970,A7060101,glutafin rolls gluten free wheat free fibre 4 ...,glutafin,1
22971,A7060101,glutafin gluten-free crisp breads 150 grams,glutafin,8


In [35]:
dmd_scripts = concise_scripts[concise_scripts['dmd_code'].isnull() == False].sort_values('dmd_code').reset_index()

In [36]:
dmd = pd.merge(dmd_scripts[['dmd_code', 'drug_name', 'name']], drug_name_counts, on='drug_name', how = "left")

In [37]:
concise = pd.merge(concise_scripts[['drug_name', 'dmd_code', 'read_2', 'bnf_code', 'name']], drug_name_counts, on='drug_name', how = "left")

In [38]:
with pd.ExcelWriter('../primary_care/medications.xlsx') as writer:  
    concise.to_excel(writer, sheet_name='all_unique_names')
    unknown_medications.to_excel(writer, sheet_name='unknown medications')
    bnf.to_excel(writer, sheet_name='bnf_codes')
    dmd.to_excel(writer, sheet_name='dmd_codes')

In [39]:
records

,eid,coded,data_provider,event_dt,read_2,read_3,value1,value2,value3,condition,...,first_record_d,first_record_diag,first_stroke_after_d,first_stroke_type,pri,sec,discrepancy,noRecords,noRecords2,event_int
0,1000255,"[[02/09/2015, cad, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2015-09-02,cad,NaT,NaN,[],"[[2015-09-02 00:00:00, cad, sec]]",False,1,1,NaN
1,1000421,"[[04/08/2010, cad, sec], [27/05/2010, stroke, ...","[3, 3]","[27/05/2010, 04/10/2010]","[nan, nan]","[XE0VK, XE2uV]","[nan, nan]","[nan, nan]","[nan, nan]","[TIA, CAD]",...,2010-05-27,tia,2010-05-27,stroke,"[[2010-05-27 00:00:00, tia, pri], [2010-10-04 ...","[[2010-05-27 00:00:00, stroke, sec], [2010-08-...",False,4,4,0.000000
2,1000466,"[[11/06/2002, cad, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2002-06-11,cad,NaT,NaN,[],"[[2002-06-11 00:00:00, cad, sec]]",False,1,1,NaN
3,1000584,"[[16/06/1997, cad, sec], [19/10/2004, afib, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1997-06-16,cad,NaT,NaN,[],"[[1997-06-16 00:00:00, cad, sec], [2004-10-19 ...",False,2,2,NaN
4,1000674,"[[31/05/2006, cad, sec]]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2006-05-31,cad,NaT,NaN,[],"[[2006-05-31 00:00:00, cad, sec]]",False,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53318,6020725,[],"[3, 3]","[01/01/1993, 18/09/2003]","[nan, nan]","[G66.., XE0VK]","[nan, nan]","[nan, nan]","[nan, nan]","[Stroke, TIA]",...,1993-01-01,stroke,2003-09-18,tia,"[[1993-01-01 00:00:00, stroke, pri], [2003-09-...",[],False,2,2,128.528307
53319,6021009,[],[3],[09/06/2016],[nan],[G5730],[nan],[nan],[nan],[Afib],...,2016-06-09,afib,NaT,NaN,"[[2016-06-09 00:00:00, afib, pri]]",[],False,1,1,NaN
53320,6021143,[],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[20/01/2005, 11/02/2005, 07/04/2005, 21/11/200...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[X00DW, X00DW, X00DW, X00DW, X00DW, X00DW, X00...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[TIA, TIA, TIA, TIA, TIA, TIA, TIA, TIA, TIA, ...",...,2005-01-20,tia,2005-02-11,tia,"[[2005-01-20 00:00:00, tia, pri], [2005-02-11 ...",[],False,19,19,0.722807
53321,6022795,[],[2],[19/01/2015],[G5730],[nan],[nan],[nan],[nan],[Afib],...,2015-01-19,afib,NaT,NaN,"[[2015-01-19 00:00:00, afib, pri]]",[],False,1,1,NaN


In [40]:
bnf

,bnf_code,drug_name,name,counts
0,01.01.01.01.00,aluminium hydroxide mixture sugar-free,aluminium,15
1,01.01.01.01.00,aluminium hydroxide 475mg capsules,aluminium,30
2,01.01.01.01.00,co-magaldrox 300mg/600mg tablets,co-magaldrox,1
3,01.01.01.01.00,maalox tc tablets (aventis pharma),maalox,8
4,01.01.01.01.00,co-magaldrox 300mg/600mg/5ml oral suspension s...,co-magaldrox,16
...,...,...,...,...
22968,A70601,glutafin select gluten-free part-baked white ...,glutafin,23
22969,A7060101,glutafin gluten-free wheat free multipurpose ...,glutafin,5
22970,A7060101,glutafin rolls gluten free wheat free fibre 4 ...,glutafin,1
22971,A7060101,glutafin gluten-free crisp breads 150 grams,glutafin,8


In [79]:
bnf[bnf['bnf_code'].str[0:2] == '10']

,bnf_code,drug_name,name,counts
12997,10.01.01.00.00,indometacin 25mg modified-release tablets,indometacin,4
12998,10.01.01.00.00,brexidol 20mg tablets (chiesi ltd),brexidol,95
12999,10.01.01.00.00,froben 50mg tablets (abbott laboratories ltd),froben,26
13000,10.01.01.00.00,lumiracoxib 100mg tablets,lumiracoxib,32
13001,10.01.01.00.00,indomax capsules 25mg [ashbourne],indomax,10
...,...,...,...,...
13960,10030250,traxam 3% gel (mercury pharma group ltd),traxam,13
13961,10030250,diclofenac 2% gel,diclofenac,5
13962,10030250,salicylic acid 2% / mucopolysaccharide polysul...,salicylic,13
13963,10030250,voltarol 1% emulgel p (novartis consumer healt...,voltarol,10


In [97]:
bnf[:18575].groupby('bnf_code').sum().shape

(3168, 1)

In [105]:
bnf[:18575].groupby('bnf_code').sum()['counts'].sum()

13476586

In [107]:
bnf[18575:].groupby('bnf_code').sum()['counts'].sum()

172789

In [87]:
dmd['dmd_code_str'] = [str(int(x)) for x in dmd['dmd_code']]

In [96]:
dmd[dmd['dmd_code_str'] != 0].groupby('dmd_code_str').sum().shape

(6503, 2)

In [104]:
dmd[dmd['dmd_code_str'] != 0].groupby('dmd_code_str').sum()['counts'].sum()

1174380

In [94]:
dmd[dmd['dmd_code_str'].str[0:2] == '24']

,dmd_code,drug_name,name,counts,dmd_code_str
4675,2.401110e+14,transiderm-nitro patches 5mg,transiderm-nitro,6,240111000001101
4676,2.401110e+14,transiderm-nitro 5 patch,transiderm-nitro,21,240111000001101
4677,2.401110e+14,transiderm-nitro 5 transdermal patches (novart...,transiderm-nitro,18,240111000001101
4678,2.402110e+14,nexium tabs 40mg,nexium,12,240211000001107
4679,2.413110e+14,vermox syr,vermox,1,241311000001109
...,...,...,...,...,...
12988,2.479050e+17,engerix b injection 20 micrograms/1 ml vial,engerix,4,247905001000027104
12989,2.479050e+17,engerix b vaccine 20micrograms/1ml,engerix,35,247905001000027104
12990,2.484650e+17,polyureth foa light-med exud + adhe bor fil dr...,polyureth,1,248465001000027104
12991,2.496550e+17,vicks sinex soother nasal spray,vicks,1,249655001000027104


In [106]:
unknown_medications

,read_2,count
0,a116.,35
1,a123.,250
2,a124.,6
3,a125.,75
4,a134.,110
...,...,...
6483,sn2q.,11
6484,sn2t.,13
6485,so24.,105
6486,so26.,2
